# <span style="color:blue">Evolutive grid search</span>

2023 Francesco Chiumento

This code is used to optimize the input parameters for Elastix. The __differential evolution__ algorithm is used to minimize the negative of the mean DICE coefficients resulting from the obtained segmentations. In each iteration, 9 parameters are allowed to vary within established bounds.

This algorithm is used on a restricted dataset of patients for which binary masks and MRIs are available. The parameters optimized at the end of the three phases can be subsequently used on datasets with a similar type of acquisition.

In [ ]:
%load_ext autoreload
%autoreload 2

## Path configuration

The images on which the optimized parameters will be tested should be placed inside the __preprocessed__ folder. Inside the __reference__ folder, there should be the reference MRI (__reference.mha__) and the respective masks for the femur (__reference_f.mha__) and femoral cartilage (__reference_fc.mha__). In the '__segmented__' folder, segmentations of the femoral cartilage for the MRIs will be created at each iteration, while in '__segmented_groundTruth__,' there are binary masks of the respective patient segmentations done manually. To set up these folders and TXT files correctly, please refer to the pyKNEEr documentation.


In [ ]:
import os
import evolutive_grid_search as egs
from scipy.optimize import differential_evolution

#Definition of the base path
base_path = os.getcwd()

## 1. Rigid optimization phase

In the first phase, only the parameters related to rigid transformation are allowed to vary, while the parameters related to the similarity and spline phases are kept fixed at their original values.

In [ ]:
# Definition of bounds and fixed parameters
bounds_rigid = [(3, 6), (1, 3), (32, 128), (1, 4), (2000, 8000), (1, 3), (1,2), (250, 2000), (3, 4)]
fixed_params_similarity = [3, 1, 32, 1, 3000, 1, 1, 250, 3]
fixed_params_spline = [5, 1, 32, 1, 2000, 1, 1, 1000, 3]
# Perform the rigid optimization
try:
    result_rigid = differential_evolution(egs.rigid_optimization_function, bounds_rigid, \
                                          args=(fixed_params_similarity, fixed_params_spline), strategy='best1bin', disp=True, polish=True)
except Exception as e:
    print(e)

## 2. Similarity optimization phase

In the second phase, the best parameters obtained in the rigid phase are kept fixed, and the parameters related to the similarity transformation are allowed to vary, while the parameters related to the spline phase are kept fixed as their original values.

In [ ]:
bounds_similarity = [(3, 6), (1, 3), (32, 128), (1, 4), (2000, 8000), (1, 3), (1,2), (250, 2000), (3, 4)]
# Use the parameters optimized during the rigid phase
optimized_params_rigid = egs.best_params_rigid
# Perform the similarity optimization
try:
    result_similarity = differential_evolution(egs.similarity_optimization_function, bounds_similarity, \
                                               args=(optimized_params_rigid, fixed_params_spline), strategy='best1bin', disp=True, polish=True)
except Exception as e:
    print(e)

## 3. Spline optimization phase

In the final phase, the best parameters obtained in the first two phases are kept fixed, while the parameters related to the spline phase are allowed to vary.

In [ ]:
bounds_spline = [(3, 6), (1, 3), (32, 128), (1, 4), (2000, 8000), (1, 3), (1,2), (250, 2000), (3, 4)]
# Use the parameters optimized in previous stages
optimized_params_similarity = egs.best_params_similarity
# Perform the spline optimization
try:
    result_spline = differential_evolution(egs.spline_optimization_function, bounds_spline, \
                                           args=(optimized_params_rigid, optimized_params_similarity), strategy='best1bin', disp=True, polish=True)
except Exception as e:
    print(e)
optimized_params_spline = egs.best_params_spline

## Statistical analysis

A statistical analysis of the results is presented. The obtained graph shows the iterations in which the best parameters were achieved.

In [ ]:
# Building the full path to the parameters_optimization.csv file
filepath = os.path.join(base_path, "pykneer", "optimal_parameters", "optimal_parameters.csv")
# Perform statistical analysis
egs.statistical_results_analysis(filepath, optimized_params_rigid, optimized_params_similarity, optimized_params_spline)

## References

[1] __Source__  
__Paper__: Bonaretti S., Gold G., Beaupre G. pyKNEEr: [*An image analysis workflow for open and reproducible research on femoral knee cartilage*](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0226501) PLOS ONE 15(1): e0226501  
__Code__: Bonaretti S. pyKNEEr. Zenodo. 2019. 10.5281/zenodo.2574171 [*Link*](https://zenodo.org/records/7695948)  
__Data__: Dataset in (Bonaretti S. et al. 2019). Zenodo. 10.5281/zenodo.2583184 [*Link*](https://zenodo.org/records/2583184)  

## Computer system details

In [ ]:
%load_ext watermark
print ("elastix 4.8.0\n")
%watermark -v -m -p pykneer,SimpleITK,numpy,matplotlib,multiprocessing,ipywidgets,watermark
print (" ")
%watermark -u -n -t -z 